In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

2023-09-22 02:11:17.110107: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-22 02:11:17.767235: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/roronoa/.conda/envs/tf/lib/python3.7/site-packages/nvidia/cudnn/lib:/home/roronoa/.conda/envs/tf/lib/:
2023-09-22 02:11:17.767321: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/roronoa/.conda/

In [2]:
df = pd.read_csv('../Data/CleanedData.csv')

In [3]:
df.head()

,subject,transformed text
0,1,donald trump wish american happi new year leav...
1,1,hous intellig committe chairman devin nune go ...
2,1,friday reveal former milwauke sheriff david cl...
3,1,christma day donald trump announc would back w...
4,1,pope franci use annual christma day messag reb...


In [4]:
texts = df['transformed text'].astype(str).tolist()
labels = df['subject'].astype(int).tolist()

In [5]:
max_sequence_length = 1000  # Set your desired sequence length
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
X = pad_sequences(sequences, maxlen=max_sequence_length)

In [6]:
y = tf.keras.utils.to_categorical(labels, num_classes=6)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_sequence_length),
    tf.keras.layers.LSTM(128, return_sequences=True, kernel_regularizer=l2(0.01)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
    tf.keras.layers.Dense(6, activation='softmax')
])

2023-09-22 02:11:26.159302: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-22 02:11:26.185404: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-22 02:11:26.185670: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-22 02:11:26.186447: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [12]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2,callbacks=[early_stopping])

Epoch 1/5
863/863 [==============================] - 75s 85ms/step - loss: 0.6109 - accuracy: 0.8409 - val_loss: 0.8735 - val_accuracy: 0.7672
Epoch 2/5
863/863 [==============================] - 54s 62ms/step - loss: 0.5539 - accuracy: 0.8463 - val_loss: 1.5851 - val_accuracy: 0.6017
Epoch 3/5
863/863 [==============================] - 44s 51ms/step - loss: 0.5092 - accuracy: 0.8553 - val_loss: 1.2748 - val_accuracy: 0.7471
Epoch 4/5
863/863 [==============================] - 42s 49ms/step - loss: 0.4551 - accuracy: 0.8626 - val_loss: 1.0504 - val_accuracy: 0.7335
Epoch 5/5
863/863 [==============================] - 41s 48ms/step - loss: 0.4450 - accuracy: 0.8677 - val_loss: 0.8454 - val_accuracy: 0.7477


In [13]:
y_pred = model.predict(X_test)

# Convert predictions to class labels
predicted_labels = np.argmax(y_pred, axis=1)

# Calculate F1 score
f1 = f1_score(np.argmax(y_test, axis=1), predicted_labels, average='weighted')

print("F1 Score:", f1)

270/270 [==============================] - 4s 12ms/step
F1 Score: 0.7500573213615905
